<a href="https://colab.research.google.com/github/Rai90/ggepy-tutorial/blob/main/ggepy_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**

First of all, run the following two cells to initialize the API. The output will contain instructions on how to grant this notebook access to Earth Engine using your account.

In [2]:
import ee

# Trigger the authentication flow.
ee.Authenticate()



In [3]:
# Initialize the library.
ee.Initialize(project='ee-raischwalbert')

In [4]:
import geemap

# **Catalog**

Here you can check the [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/catalog).


I will leave here some ag useful datasets you may have interest:


### Satellite
*  [MODIS 8-day 250m](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD09Q1)
*   [MODIS Land Surface Temperature 1km](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MYD11A1)
*  [Sentinel 2 SR](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED)
*   [Landsat 9](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC09_C02_T1_L2)


### Weather
*  [MODIS 8-day 250m](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD09Q1)
*   [MODIS Land Surface Temperature 1km](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MYD11A1)
*  [Sentinel 2 SR](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED)
*   [Landsat 9](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC09_C02_T1_L2)





# **Accessing and navegating within collection**



In [6]:
a = ee.ImageCollection("USDA/NASS/CDL")

In [7]:
print(a.getInfo().keys())

dict_keys(['type', 'bands', 'version', 'id', 'properties', 'features'])


In [8]:
#images: get imagecollection size

print(a.size().getInfo())


29


In [9]:
# prompt: from a, print all ids of each feature

for image in a.getInfo()['features']:
  print(image['id'])


USDA/NASS/CDL/1997
USDA/NASS/CDL/1998
USDA/NASS/CDL/1999
USDA/NASS/CDL/2000
USDA/NASS/CDL/2001
USDA/NASS/CDL/2002
USDA/NASS/CDL/2003
USDA/NASS/CDL/2004
USDA/NASS/CDL/2005a
USDA/NASS/CDL/2005b
USDA/NASS/CDL/2006
USDA/NASS/CDL/2007a
USDA/NASS/CDL/2007b
USDA/NASS/CDL/2008
USDA/NASS/CDL/2009
USDA/NASS/CDL/2010
USDA/NASS/CDL/2011
USDA/NASS/CDL/2012
USDA/NASS/CDL/2013
USDA/NASS/CDL/2014
USDA/NASS/CDL/2015
USDA/NASS/CDL/2016
USDA/NASS/CDL/2017
USDA/NASS/CDL/2018
USDA/NASS/CDL/2019
USDA/NASS/CDL/2020
USDA/NASS/CDL/2021
USDA/NASS/CDL/2022
USDA/NASS/CDL/2023


# **Select an image from collection**

In [10]:
cdl2023 = ee.Image("USDA/NASS/CDL/2023")

for image in cdl2023.getInfo()['bands']:
  print(image['id'])


cropland
cultivated
confidence


In [11]:
cdl2023 = cdl2023.select(["cropland"])

In [30]:
# prompt: add cdl2023 to map using ggemap

m = geemap.Map()
m.add_basemap("HYBRID")
m.setCenter(-98.5, 38.5, 5)
m.addLayer(cdl2023, {}, "CDL 2023")
m


Map(center=[38.5, -98.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

# **Store polygon drawed over map**

In [31]:
polygon = ee.FeatureCollection(m.draw_last_feature)
polygon

In [32]:
clipped_image = cdl2023.clip(polygon)

In [33]:
mm = geemap.Map()
mm.add_basemap("HYBRID")
mm.addLayer(clipped_image)
mm

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# **Filtering**

In [34]:
# prompt: get daymet data - filter date and filter bounds

daymet = ee.ImageCollection('NASA/ORNL/DAYMET_V4').filterDate("2021-01-01","2022-01-01") \
.filterBounds(polygon)

print("amount of images = ",daymet.size().getInfo())

for band in ee.Image(daymet.first()).getInfo()["bands"]:
  print(band["id"], end=" ")



amount of images =  365
dayl prcp srad swe tmax tmin vp 

# Within the polygon, check soybean areas where august cumulative rain was higher then 100 mm

In [46]:
AUG_daymet = daymet.filter(ee.Filter.calendarRange(8, 8, 'month'))

print("n = " ,AUG_daymet.size().getInfo())

AUG_daymet = AUG_daymet.select(["prcp"]).sum().clip(polygon)

AUG_daymet_mask = AUG_daymet.gte(100)

#5 means soybean on cdl
soybean_mask = cdl2023.select(["cropland"]).eq(5)


AUG_daymet = AUG_daymet.updateMask(AUG_daymet_mask).updateMask(soybean_mask)

n =  31


In [50]:
vis_params = {
    'min': 120,
    'max': 160,
    'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000'],
    'bands': ['prcp']
}


m2 = geemap.Map()
m2.add_basemap("HYBRID")
m2.addLayer(AUG_daymet,vis_params)
#m2.addLayer(polygon)
m2



Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…